In [1]:
import os

os.environ["OE_LICENSE"] = "/home/jsetiadi/oe_license.txt"

import simtk.unit as unit
from openeye import oechem, oeomega, oequacpac
from openff.evaluator.datasets.taproom import TaproomDataSet
from openff.evaluator.forcefield import SmirnoffForceFieldSource
from openff.toolkit.topology import Molecule
from openff.toolkit.typing.engines import smirnoff
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler
from openff.toolkit.utils import DEFAULT_AROMATICITY_MODEL
from pkg_resources import resource_filename

In [2]:
def add_molecule_to_library_charge(library_charge_handler: LibraryChargeHandler, molecule: Molecule):
    # Create a smiles pattern which assign every atom a unique index.
    mapped_smiles = molecule.to_smiles(mapped=True)

    # Build a library charge handler for the molecule.
    library_charge_handler.add_parameter(
        parameter_kwargs={
            "smirks": mapped_smiles,
            **{
                f"charge{i + 1}": charge
                for i, charge in enumerate(molecule.partial_charges)
            },
        }
    )

In [3]:
def charge_guest_molecule(guest_molecule: Molecule) -> Molecule:
    """Charge a guest molecules using openeye AM1BCC charge model.
    """
    oe_molecule: oechem.OEMol = guest_molecule.to_openeye()

    # Generate the conformers.
    omega = oeomega.OEOmega()
    omega.SetMaxConfs(100)
    omega.SetIncludeInput(False)
    omega.SetStrictStereo(True)
    omega(oe_molecule)
    assert omega.Build(oe_molecule) == oeomega.OEOmegaReturnCode_Success

    # Assign the charges
    charge_engine = oequacpac.OEAM1BCCELF10Charges()
    assert oequacpac.OEAssignCharges(oe_molecule, charge_engine)

    # Map the OE molecule back to an OpenFF one.
    off_molecule = Molecule.from_openeye(oe_molecule)
    
    return off_molecule

In [4]:
def charge_host_molecule(host_molecule: Molecule) -> Molecule:
    """Charge a macrocycle host molecules using a combination of the
    ``OEMacrocycleOmega`` varient of Omega and the default openeye AM1BCC charge
    model.
    """
    oe_molecule: oechem.OEMol = host_molecule.to_openeye()

    # Enable macrocycle sampling of conformers
    omega_options = oeomega.OEMacrocycleOmegaOptions()

    # Generate the conformers.
    omega = oeomega.OEMacrocycleOmega(omega_options)
    assert omega.Build(oe_molecule) == oeomega.OEOmegaReturnCode_Success

    # Assign the charges
    charge_engine = oequacpac.OEAM1BCCELF10Charges()
    assert oequacpac.OEAssignCharges(oe_molecule, charge_engine)

    # Map the OE molecule back to an OpenFF one.
    off_molecule = Molecule.from_openeye(oe_molecule)
    
    return off_molecule

## Collect info of Taproom dataset

In [5]:
data_set = TaproomDataSet(
    in_vacuum=True,
)
all_smiles = set(
    component.smiles
    for substance in data_set.substances
    for component in substance.components
)
guest_smiles = [smile for smile in all_smiles if len(smile)<100]
host_smiles = [smile for smile in all_smiles if len(smile)>100]

### Create Library Charge

In [6]:
library_charge_handler = LibraryChargeHandler(version=0.3)

### Charge guest molecules

In [7]:
for id, smiles in enumerate(guest_smiles):
    print(f"{id} - Charging molecule: {smiles}")
    guest_uncharged = Molecule.from_smiles(smiles)
    guest_charged = charge_guest_molecule(guest_uncharged)
    
    add_molecule_to_library_charge(library_charge_handler, guest_charged)
    
    total_charge = guest_charged.partial_charges.sum().value_in_unit(unit.elementary_charge)

0 - Charging molecule: CC(C)CCC(=O)[O-]
1 - Charging molecule: CCCCCC[N+](C)(C)C
2 - Charging molecule: c1ccc(cc1)C(=O)[O-]
3 - Charging molecule: C[C@H](C12CC3CC(C1)CC(C3)C2)[NH3+]
4 - Charging molecule: C[C@H](CC(=O)[O-])c1ccccc1
5 - Charging molecule: C[NH2+][C@]1(CCCCC1=O)c2ccccc2Cl
6 - Charging molecule: CCCCC(=O)[O-]
7 - Charging molecule: CCCCCCCC(=O)[O-]
8 - Charging molecule: C1CCC(C1)CC(=O)[O-]
9 - Charging molecule: C1CCCCCC(CCCCC1)[NH3+]
10 - Charging molecule: C[N@H+]1CC[C@]23c4c5ccc(c4O[C@H]2[C@H](C=C[C@H]3[C@H]1C5)O)O
11 - Charging molecule: CCc1ccc(cc1)C(=O)[O-]
12 - Charging molecule: C1CCCC(CCC1)O
13 - Charging molecule: CC(C)(CC=C)C(=O)[O-]
14 - Charging molecule: c1cc2c3c(c1)C(=O)N(C[C@@H]3CCC2)[C@@H]4C[NH+]5CCC4CC5
15 - Charging molecule: C[N@H+]1CC[C@]23c4c5ccc(c4O[C@H]2C(=O)CC[C@H]3[C@H]1C5)O
16 - Charging molecule: COc1cc2c(cc1OC)C(=O)[C@@H](C2)CC3CC[NH+](CC3)Cc4ccccc4
17 - Charging molecule: CCCC[NH2+]C
18 - Charging molecule: C[C@H](CC=C)C(=O)[O-]
19 - Chargin

### Charge host molecules

In [9]:
host_mol2 = []
for data in data_set.properties:
    mol2_path = data.metadata["host_file_paths"]["host_mol2_path"]
    if mol2_path not in host_mol2:
        host_mol2.append(mol2_path)

In [10]:
for mol2 in host_mol2:
    print(f"Charging molecule: {mol2}")
    host_uncharged = Molecule.from_file(mol2)
    host_charged = charge_host_molecule(guest_uncharged)
    #host_charged = Molecule.from_file(mol2)
    smiles = host_charged.to_smiles()
    
    add_molecule_to_library_charge(library_charge_handler, host_charged)
    
    total_charge = host_charged.partial_charges.sum().value_in_unit(unit.elementary_charge)

Charging molecule: -8.00 [H]c1c(c(c2c(c1Oc3c(c4c(c(c3[H])Oc5c(c(c(c(c5[H])Oc6c(c7c(c(c6[H])Oc8c(c(c(c(c8[H])Oc9c(c1c(c(c9[H])Oc3c(c(c(c(c3[H])Oc3c(c(c(c(c3[H])O2)[H])C2(Oc3c(c5c6c(c3C(c3c(c8c(c(c3O2)[H])OC1(Oc1c(c2c(c(c1C8([H])C([H])([H])C([H])([H])C(=O)[O-])[H])C(c1c(c(c(c(c1OC7(O2)[H])[H])OC4(O5)[H])C6([H])C([H])([H])C([H])([H])C(=O)[O-])[H])([H])C([H])([H])C([H])([H])C(=O)[O-])[H])[H])[H])([H])C([H])([H])C([H])([H])C(=O)[O-])[H])[H])[H])[H])[H])C(=O)[O-])[H])[H])[H])[H])C(=O)[O-])[H])[H])[H])[H])C(=O)[O-])[H])[H])[H])[H])[H])C(=O)[O-]
Charging molecule: -0.00 [H][C@@]1([C@]2([C@@](O[C@@]([C@]1([H])O[H])(O[C@]3([C@]([C@@]([C@@](O[C@]4([C@]([C@@]([C@@](O[C@]5([C@]([C@@]([C@@](O[C@]6([C@]([C@@]([C@@](O[C@]7([C@]([C@@]([C@@](O[C@]8([C@]([C@@]([C@@](O2)(O[C@]8([H])C([H])([H])O[H])[H])([H])O[H])([H])O[H])[H])(O[C@]7([H])C([H])([H])O[H])[H])([H])O[H])([H])O[H])[H])(O[C@]6([H])C([H])([H])O[H])[H])([H])O[H])([H])O[H])[H])(O[C@]5([H])C([H])([H])O[H])[H])([H])O[H])([H])O[H])[H])(O[C@]4([H])C([

### Save Charges to file

In [11]:
force_field = ForceField(aromaticity_model=DEFAULT_AROMATICITY_MODEL)
force_field.register_parameter_handler(library_charge_handler)
force_field.to_file("taproom_AM1BCCELF10_charges.offxml")